In [8]:
import random
import math
import copy
import numpy as np

# generate random samples (for received vectors) assuming transmitting all zero input
# the current case is for BSC channel
# change get_lch for other channels
def gen_rec(p, length):
    x = []
    for _ in range(length):
        rand_num = random.random()
        if (rand_num<p):
            x.append(1)
        else:
            x.append(0)
    print("error weight = ",sum(x))
    return x

# calculate initial channel llrs
# the current case is for BSC channel
# change get_lch for other channels
def get_lch(p, length):
    lch = []
    if p == 0:
        llr = 22.0
    else:
        llr = math.log((1 - p) / p)
    for _ in range(length):
        lch.append(llr)
    return lch

# for CN-to-VN update
def phi(x):
    t = 0.0
    if x > 27.5:
        t = 0.0  # You can replace this line with t = 2 * math.exp(-x) if needed
    elif x < 1e-12:
        t = 28.0  # You can replace this line with t = -math.log(x/2) if needed
    else:
        t = -math.log(math.tanh(x / 2))
    return t

class Node:
    def __init__(self, neighbors=None, message=None):
        self.neighbors = []
        self.message = []
        if neighbors!=None:
            self.neighbors = copy.deepcopy(neighbors)
        if self.message!=None:
            self.message = copy.deepcopy(message)

    def print_neighbors(self):
        print(",".join(map(str, self.neighbors)))

class VN(Node):
    # VN to CN update at a single VN
    def update_v2c(self, lh):
        sum_lh = lh + sum(self.message)
        for i in range(len(self.message)):
            self.message[i] = sum_lh-self.message[i]
        return sum_lh

class CN(Node):
    # transform input llrs on domain of real number R (call it L domain) into domain (GF(2),R) (call it D domain)
    def l2d(self):
        sign = []
        for i in range(len(self.message)):
            llr = self.message[i]
            sign.append((llr<0.0))
            self.message[i] = phi(abs(llr))
        return sign
    
    # transform output llrs on domain (GF(2),R) (call it D domain) into domain of real number R (call it L domain)
    def d2l(self, sign):
        for i in range(len(self.message)):
            llr = self.message[i]
            if sign[i]:
                self.message[i] = -phi(llr)
            else:
                self.message[i] = phi(llr)
                
    # CN to VN update at a single CN
    def update_c2v(self, syndrome):
        sign = self.l2d()
        sum_msg = sum(self.message)
        sign_sum = (syndrome!=0)
        #if (syndrome!=0):
        #    for i in range(len(sign)):
        #        print(sign[i],",")
        for i in range(len(self.message)):
            if sign[i]:
                sign_sum = not sign_sum
        #if (syndrome!=0):
        #    print("sign_sum = ",sign_sum)
        for i in range(len(self.message)):
            self.message[i] = sum_msg - self.message[i]
            if sign[i]:
                sign[i] = not sign_sum
            else:
                sign[i] = sign_sum
        #if (syndrome!=0):
        #    for i in range(len(sign)):
        #        print(sign[i],",")
        self.d2l(sign)
        #for i in range(len(self.message)):
        #    if self.message[i]<0:
        #        print("error !")
        #        break


In [33]:
class Code:
    def __init__(self, H):
        self.vn = [] # stores the adjacent edge list adjacent to each VN
        self.cn = [] # stores the adjacent edge list adjacent to each CN
        self.vc = [] # stores the adjacent VN list adjacent to each CN
        
        num_cn = len(H)
        num_vn = len(H[0])
        num_e = 0
        Ht = [list(row) for row in H]
        for c in range(num_cn):
            neigh = []
            neigh_c = []
            for v in range(num_vn):
                if H[c][v]:
                    neigh.append(num_e)
                    neigh_c.append(v)
                    Ht[c][v] = num_e
                    num_e += 1
            self.vc.append(neigh_c)
            mes = [0.0] * len(neigh)
            cn_node = CN(neigh, mes)
            self.cn.append(cn_node)

        for v in range(num_vn):
            neigh = []
            for c in range(num_cn):
                if H[c][v]:
                    neigh.append(Ht[c][v])
            mes = [0.0] * len(neigh)
            vn_node = VN(neigh, mes)
            self.vn.append(vn_node)

        self.lv2c = [0.0] * num_e # stores the VN to CN LLRs at each iteration
        self.lc2v = [0.0] * num_e # stores the CN to VN LLRs at each iteration

    # calculate the syndrome of received vector x
    def get_syndrome(self, x):
        syndrome = []
        for c in range(len(self.vc)):
            parity = 0
            for v in self.vc[c]:
                parity ^= x[v]
            syndrome.append(parity)
            #if (parity):
            #    print("syndrome =1 at CN ",c)
        return syndrome

    # VN-to-CN update at each iteration
    def update_v2c(self, lch):
        lh = [0.0] * len(self.vn)
        for n in range(len(self.vn)):
            lc = self.vn[n].neighbors
            for c in range(len(lc)):
                self.vn[n].message[c]=self.lc2v[lc[c]]
            lh[n] = self.vn[n].update_v2c(lch[n])
            for c in range(len(lc)):
                self.lv2c[lc[c]] = self.vn[n].message[c]
            #if (n==1):
            #print(n,",".join(map(str, self.vn[n].message)))
        return lh

    # CN-to-VN update at each iteration
    def update_c2v(self, syndrome):
        for n in range(len(self.cn)):
            lv = self.cn[n].neighbors
            for v in range(len(lv)):
                self.cn[n].message[v]=self.lv2c[lv[v]]
            self.cn[n].update_c2v(syndrome[n])
            for v in range(len(lv)):
                self.lc2v[lv[v]] = self.cn[n].message[v]
            #if (syndrome[n]):
            #print(n,",".join(map(str, self.cn[n].message)))

    def decode(self, x, lch, num_iter):
        syndrome = self.get_syndrome(x)
        satisfy=True
        estimate=x
        for k in range(num_iter):
            lh = self.update_v2c(lch)
            self.update_c2v(syndrome)
            for v in range(len(self.vn)):
                if (lh[v]<0):
                    estimate[v]=1
                else:
                    estimate[v]=0
            satisfy=True
            for c in range(len(self.vc)):
                parity = 0
                for v in self.vc[c]:
                    parity ^= estimate[v]
                if parity!=syndrome[c]:
                    satisfy = False
                    break
            if satisfy:
                print("Stop at iteration ",k+1)
                break
                
        success_decode = True
        if satisfy: 
            for v in range(len(self.vn)):
                if (x[v] != estimate[v]):
                    success_decode = False
                    print("Decoder failed: satisfied")
                    break
            if success_decode:
                print("Decoder succeed")
        else:
            success_decode=False
            print("Decoder failed: unsatisfied")
            
        return success_decode

In [34]:
def ldpc_simulation(H, p, num_samples, num_iter):
    ldpc = Code(H)
    num_vn = len(H[0])
    num_fail = 0

    for k in range(num_samples):
        print(k,"'th sample: ")
        x = gen_rec(p, num_vn)
        # print(', '.join(map(str, x)))
        lch = get_lch(p, num_vn)
        dec_suc = ldpc.decode(x, lch, num_iter)
        
        if not dec_suc:
            num_fail += 1

    FER = float(num_fail) / float(num_samples)
    return FER


In [35]:
def read_matrix_from_file(file_path):
    try:
        # Initialize an empty list to store the matrix
        H = []

        # Open the file and read its contents
        with open(file_path, 'r') as file:
            for line in file:
                # Split each line into individual values and convert them to integers
                row = [int(value) for value in line.split()]
                H.append(row)

        return H

    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None

# Example usage:
file_path = 'ParityCheckMatrices/SC_3_3_2_3_2.txt'  # Replace with the actual path to your text file
matrix = read_matrix_from_file(file_path)
if matrix is not None:
    for row in matrix:
        print(row)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 

In [39]:
#file_path = 'GD_4_24_6_17_40.txt'
file_path='ParityCheckMatrices/GD_3_7_5_13_30.txt'
H = read_matrix_from_file(file_path) # the parity check matrix
p=0.073 # the crossover probability in BSC(p)
num_samples=300 # number of samples being tested
num_iter=50 # upper limit of the number of iteration
fer=ldpc_simulation(H, p, num_samples, num_iter) # frame error rate/block error rate

print("fer=",fer)

0 'th sample: 
error weight =  231
Decoder failed: unsatisfied
1 'th sample: 
error weight =  179
Stop at iteration  26
Decoder succeed
2 'th sample: 
error weight =  184
Stop at iteration  17
Decoder succeed
3 'th sample: 
error weight =  203
Stop at iteration  43
Decoder succeed
4 'th sample: 
error weight =  208
Stop at iteration  27
Decoder succeed
5 'th sample: 
error weight =  162
Stop at iteration  14
Decoder succeed
6 'th sample: 
error weight =  198
Stop at iteration  32
Decoder succeed
7 'th sample: 
error weight =  198
Decoder failed: unsatisfied
8 'th sample: 
error weight =  197
Stop at iteration  32
Decoder succeed
9 'th sample: 
error weight =  196
Stop at iteration  24
Decoder succeed
10 'th sample: 
error weight =  205
Stop at iteration  26
Decoder succeed
11 'th sample: 
error weight =  205
Stop at iteration  38
Decoder succeed
12 'th sample: 
error weight =  204
Decoder failed: unsatisfied
13 'th sample: 
error weight =  201
Stop at iteration  21
Decoder succeed
14 '

Stop at iteration  49
Decoder succeed
117 'th sample: 
error weight =  216
Decoder failed: unsatisfied
118 'th sample: 
error weight =  210
Decoder failed: unsatisfied
119 'th sample: 
error weight =  208
Decoder failed: unsatisfied
120 'th sample: 
error weight =  181
Stop at iteration  16
Decoder succeed
121 'th sample: 
error weight =  193
Decoder failed: unsatisfied
122 'th sample: 
error weight =  193
Stop at iteration  33
Decoder succeed
123 'th sample: 
error weight =  196
Stop at iteration  33
Decoder succeed
124 'th sample: 
error weight =  210
Decoder failed: unsatisfied
125 'th sample: 
error weight =  208
Decoder failed: unsatisfied
126 'th sample: 
error weight =  195
Stop at iteration  22
Decoder succeed
127 'th sample: 
error weight =  192
Stop at iteration  25
Decoder succeed
128 'th sample: 
error weight =  194
Decoder failed: unsatisfied
129 'th sample: 
error weight =  204
Stop at iteration  34
Decoder succeed
130 'th sample: 
error weight =  205
Stop at iteration  3

Stop at iteration  18
Decoder succeed
232 'th sample: 
error weight =  217
Decoder failed: unsatisfied
233 'th sample: 
error weight =  201
Decoder failed: unsatisfied
234 'th sample: 
error weight =  230
Decoder failed: unsatisfied
235 'th sample: 
error weight =  198
Stop at iteration  23
Decoder succeed
236 'th sample: 
error weight =  209
Stop at iteration  41
Decoder succeed
237 'th sample: 
error weight =  182
Stop at iteration  26
Decoder succeed
238 'th sample: 
error weight =  201
Stop at iteration  22
Decoder succeed
239 'th sample: 
error weight =  210
Decoder failed: unsatisfied
240 'th sample: 
error weight =  214
Decoder failed: unsatisfied
241 'th sample: 
error weight =  179
Stop at iteration  40
Decoder succeed
242 'th sample: 
error weight =  215
Decoder failed: unsatisfied
243 'th sample: 
error weight =  194
Stop at iteration  24
Decoder succeed
244 'th sample: 
error weight =  196
Stop at iteration  23
Decoder succeed
245 'th sample: 
error weight =  211
Decoder fa

In [99]:
x=1.5
print("phi(",x,")=",phi(x))
print("phi(phi(",x,"))=",phi(phi(x)))

phi( 1.5 )= 0.45389573690820645
phi(phi( 1.5 ))= 1.5


In [32]:
H=np.array([[1,0,0,0,1,1,1],[0,1,0,1,0,1,1],[0,0,1,1,1,0,1]])
ldpc = Code(H)
num_vn = len(H[0])
x = np.array([0,0,0,1,0,0,1])
num_iter=2
p=0.1
# print(', '.join(map(str, x)))
lch = get_lch(p, num_vn)
dec_suc = ldpc.decode(x, lch, num_iter)


0 2.1972245773362196
1 2.1972245773362196
2 2.1972245773362196
3 2.1972245773362196,2.1972245773362196
4 2.1972245773362196,2.1972245773362196
5 2.1972245773362196,2.1972245773362196
6 2.1972245773362196,2.1972245773362196,2.1972245773362196
0 -1.1308731508863314,-1.1308731508863314,-1.1308731508863314,-1.1308731508863314
1 1.1308731508863314,1.1308731508863314,1.1308731508863314,1.1308731508863314
2 1.1308731508863314,1.1308731508863314,1.1308731508863314,1.1308731508863314
0 2.1972245773362196
1 2.19722457733622
2 2.19722457733622
3 3.3280977282225512,3.3280977282225512
4 3.3280977282225512,1.0663514264498881
5 3.3280977282225512,1.0663514264498881
6 4.4589708791088825,2.19722457733622,2.19722457733622
0 -2.4872805791086137,-1.8470852228428023,-1.8470852228428023,-1.7076100746054284
1 0.7611889165572873,0.645951489223671,1.3728694438862445,0.7611889165572873
2 0.7611889165572873,0.645951489223671,1.3728694438862445,0.7611889165572873
Decoder failed: unsatisfied
